In [1]:
import onnxruntime as nxrun
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torchaudio
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier

In [7]:
dpn68b_train_df = pd.read_csv("../../production/dpn68b/v1.train.csv")
tf_efficientnet_b0_ns_train_df = pd.read_csv(
    "../../production/tf_efficientnet_b0_ns/v1.train.csv"
)
efficientnet_b3_train_df = pd.read_csv("../../production/efficientnet_b3/v1.train.csv")


dpn68b_val_df = pd.read_csv("../../production/dpn68b/v1.val.csv")
tf_efficientnet_b0_ns_val_df = pd.read_csv(
    "../../production/tf_efficientnet_b0_ns/v1.val.csv"
)
efficientnet_b3_val_df = pd.read_csv("../../production/efficientnet_b3/v1.val.csv")

In [8]:
# arhmax of first row
dpn68b_train_df.iloc[:, 1:].idxmax(axis=1)

0         litegr
1         spodov
2         comsan
3        eaywag1
4         greegr
          ...   
58696     grnsan
58697     revbul
58698    grewar3
58699    asikoe2
58700     rorpar
Length: 58701, dtype: object

In [11]:
# function to modify df and  apply softmax to a row
def apply_softmax(df):
    df.iloc[:, 1:] = df.iloc[:, 1:].apply(
        lambda x: np.exp(x) / np.sum(np.exp(x)), axis=1
    )
    return df

In [12]:
# apply on each row softmax but exclude y column
dpn68b_train_df = apply_softmax(dpn68b_train_df)
tf_efficientnet_b0_ns_train_df = apply_softmax(tf_efficientnet_b0_ns_train_df)
efficientnet_b3_train_df = apply_softmax(efficientnet_b3_train_df)

dpn68b_val_df = apply_softmax(dpn68b_val_df)
tf_efficientnet_b0_ns_val_df = apply_softmax(tf_efficientnet_b0_ns_val_df)
efficientnet_b3_val_df = apply_softmax(efficientnet_b3_val_df)

In [13]:
# convert y column to int
dpn68b_train_df["y"] = dpn68b_train_df["y"].astype(int)
tf_efficientnet_b0_ns_train_df["y"] = tf_efficientnet_b0_ns_train_df["y"].astype(int)
efficientnet_b3_train_df["y"] = efficientnet_b3_train_df["y"].astype(int)
dpn68b_val_df["y"] = dpn68b_val_df["y"].astype(int)
tf_efficientnet_b0_ns_val_df["y"] = tf_efficientnet_b0_ns_val_df["y"].astype(int)
efficientnet_b3_val_df["y"] = efficientnet_b3_val_df["y"].astype(int)

In [14]:
# stack dataframes together wihoout y culumn
train_df = pd.concat(
    [dpn68b_train_df, tf_efficientnet_b0_ns_train_df, efficientnet_b3_train_df], axis=1
)
train_df = train_df.drop(columns=["y"])
train_y = dpn68b_train_df["y"]

val_df = pd.concat(
    [dpn68b_val_df, tf_efficientnet_b0_ns_val_df, efficientnet_b3_val_df], axis=1
)
val_df = val_df.drop(columns=["y"])
val_y = dpn68b_val_df["y"]

In [15]:
train_df.columns.tolist()

['asbfly',
 'ashdro1',
 'ashpri1',
 'ashwoo2',
 'asikoe2',
 'asiope1',
 'aspfly1',
 'aspswi1',
 'barfly1',
 'barswa',
 'bcnher',
 'bkcbul1',
 'bkrfla1',
 'bkskit1',
 'bkwsti',
 'bladro1',
 'blaeag1',
 'blakit1',
 'blhori1',
 'blnmon1',
 'blrwar1',
 'bncwoo3',
 'brakit1',
 'brasta1',
 'brcful1',
 'brfowl1',
 'brnhao1',
 'brnshr',
 'brodro1',
 'brwjac1',
 'brwowl1',
 'btbeat1',
 'bwfshr1',
 'categr',
 'chbeat1',
 'cohcuc1',
 'comfla1',
 'comgre',
 'comior1',
 'comkin1',
 'commoo3',
 'commyn',
 'compea',
 'comros',
 'comsan',
 'comtai1',
 'copbar1',
 'crbsun2',
 'cregos1',
 'crfbar1',
 'crseag1',
 'dafbab1',
 'darter2',
 'eaywag1',
 'emedov2',
 'eucdov',
 'eurbla2',
 'eurcoo',
 'forwag1',
 'gargan',
 'gloibi',
 'goflea1',
 'graher1',
 'grbeat1',
 'grecou1',
 'greegr',
 'grefla1',
 'grehor1',
 'grejun2',
 'grenig1',
 'grewar3',
 'grnsan',
 'grnwar1',
 'grtdro1',
 'gryfra',
 'grynig2',
 'grywag',
 'gybpri1',
 'gyhcaf1',
 'heswoo1',
 'hoopoe',
 'houcro1',
 'houspa',
 'inbrob1',
 'indpit1',
 

In [16]:
meta_features_train = train_df.to_numpy()
y_train = train_y.to_numpy()

meta_features_val = val_df.to_numpy()
y_val = val_y.to_numpy()

In [17]:
meta_model = CatBoostClassifier(
    iterations=1000, learning_rate=0.01, depth=6, verbose=0, task_type="GPU"
)

In [93]:
# load model
meta_model.load_model("meta_model.cbm")

In [18]:
meta_model.fit(meta_features_train, y_train)

In [19]:
# save meta_model
meta_model.save_model("meta_model.cbm")

In [21]:
meta_predictions = meta_model.predict(meta_features_val)
accuracy = accuracy_score(y_val, meta_predictions)

In [22]:
accuracy

0.6800899427636958

In [96]:
min(meta_predictions)

array([0])

In [29]:
class MetaModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MetaModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

In [24]:
# add rows from  train to tests df
all_data_df = pd.concat([train_df, val_df], axis=0)
all_data_y = np.concatenate([y_train, y_val])
X_train, X_test, y_train, y_test = train_test_split(
    all_data_df.to_numpy(), all_data_y, test_size=0.2, random_state=42
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

In [33]:
input_size = X_train.shape[1]  # Number of meta-features
hidden_size = 128
output_size = len(np.unique(all_data_y))  # Number of classes

model = MetaModel(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [38]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [39]:
# Training the model
num_epochs = 3000
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [2/3000], Loss: 1.0850
Epoch [4/3000], Loss: 1.0824
Epoch [6/3000], Loss: 1.0855
Epoch [8/3000], Loss: 1.0842
Epoch [10/3000], Loss: 1.0848
Epoch [12/3000], Loss: 1.0882
Epoch [14/3000], Loss: 1.0844
Epoch [16/3000], Loss: 1.0884
Epoch [18/3000], Loss: 1.0854
Epoch [20/3000], Loss: 1.0835
Epoch [22/3000], Loss: 1.0870
Epoch [24/3000], Loss: 1.0790
Epoch [26/3000], Loss: 1.0862
Epoch [28/3000], Loss: 1.0862
Epoch [30/3000], Loss: 1.0857
Epoch [32/3000], Loss: 1.0811
Epoch [34/3000], Loss: 1.0826
Epoch [36/3000], Loss: 1.0828
Epoch [38/3000], Loss: 1.0866
Epoch [40/3000], Loss: 1.0887
Epoch [42/3000], Loss: 1.0858
Epoch [44/3000], Loss: 1.0868
Epoch [46/3000], Loss: 1.0855
Epoch [48/3000], Loss: 1.0800
Epoch [50/3000], Loss: 1.0854
Epoch [52/3000], Loss: 1.0871
Epoch [54/3000], Loss: 1.0876
Epoch [56/3000], Loss: 1.0821
Epoch [58/3000], Loss: 1.0847
Epoch [60/3000], Loss: 1.0833
Epoch [62/3000], Loss: 1.0843
Epoch [64/3000], Loss: 1.0835
Epoch [66/3000], Loss: 1.0877
Epoch [68/3000

In [99]:
model.load_state_dict(torch.load("meta_model.pth"))

<All keys matched successfully>

In [42]:
# save jit model
model = model.to("cpu")
torch.jit.save(torch.jit.script(model), "meta_model.pt")

In [41]:
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = accuracy_score(y_test.cpu(), predicted.cpu())
    print(f"Ensemble Model Accuracy: {accuracy:.4f}")

Ensemble Model Accuracy: 0.8402


In [105]:
# softmax
softmax = nn.Softmax(dim=1)
outputs1 = softmax(outputs)
outputs1[0]

tensor([2.4179e-09, 1.5859e-08, 4.9914e-09, 1.0417e-08, 5.3423e-08, 2.8951e-09,
        3.8859e-09, 1.0728e-08, 1.2620e-10, 8.3270e-06, 5.5221e-05, 1.9113e-11,
        1.5181e-08, 2.2436e-06, 9.9454e-01, 8.7898e-07, 2.4719e-09, 4.9289e-06,
        2.5505e-09, 2.0129e-08, 1.2814e-06, 4.1809e-09, 1.5721e-07, 1.9579e-09,
        1.3403e-08, 1.0716e-08, 8.5534e-09, 3.9499e-07, 5.9987e-08, 1.9226e-08,
        8.0016e-09, 1.3039e-07, 1.9754e-10, 6.1245e-04, 3.9653e-09, 2.1754e-08,
        3.6285e-09, 5.0791e-05, 3.3474e-07, 1.9684e-06, 2.1195e-04, 2.7917e-06,
        1.9002e-07, 3.9435e-06, 4.4361e-06, 1.5169e-08, 8.9984e-08, 1.0855e-10,
        3.2561e-07, 9.3178e-09, 1.2788e-07, 4.4158e-09, 9.3306e-09, 6.3601e-05,
        3.9292e-11, 1.3248e-06, 6.6989e-11, 6.2909e-06, 3.5056e-09, 1.8391e-04,
        1.1768e-05, 4.3406e-11, 5.0470e-05, 3.7790e-07, 3.0548e-08, 2.8209e-04,
        2.1308e-10, 1.4738e-08, 1.8641e-09, 1.0149e-10, 2.7075e-07, 7.4253e-05,
        9.5298e-09, 2.1444e-07, 1.7783e-

In [109]:
sum(outputs1[0])

tensor(1.0000, device='cuda:0')

In [73]:
# save model
torch.save(model.state_dict(), "meta_model.pth")

### Prepare paths

In [ ]:
test_df = pd.DataFrame(test_paths, columns=["filepath"])
test_df.head()

In [3]:
meta_model = LogisticRegression()
meta_model.fit(meta_features_train, y_train)

LogisticRegression()

In [4]:
meta_predictions = meta_model.predict(meta_features_train)

In [5]:
meta_predictions

array([3, 3, 1, 1, 1, 1, 1, 1, 1, 1])